In [1]:
import sys
sys.path.append('../')

In [2]:
import os
import time
import rich
import numpy as np
import json
from easydict import EasyDict as edict

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from models.densenet import UnimodalDenseNet
from datasets.mri import MRI, MRIProcessor
from tasks.unimodal import Classification


In [58]:
local_rank = 0
model_checkpoint_dir = '../checkpoints/mri/densenet/'

In [80]:
model_hash = '2022-05-14_18-26-51'# '2022-05-17_03-02-22' #'2022-05-14_18-26-51'

In [81]:
model_checkpoint_path = model_checkpoint_dir + model_hash

In [82]:
config = os.path.join(model_checkpoint_path, 'configs.json')
with open(config, 'r') as f:
    config = json.load(f)
    config = edict(config)

In [83]:
np.random.seed(config.random_state)
torch.manual_seed(config.random_state)

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.allow_tf32 = True

torch.cuda.set_device(local_rank)

In [84]:
network = UnimodalDenseNet(in_channels=1,
                           out_channels=2,
                           init_features=config.init_features,
                           growth_rate=config.growth_rate,
                           block_config=tuple(config.block_config),
                           bn_size=config.bn_size,
                           dropout_rate=config.dropout_rate)

In [85]:
from monai.transforms import (
    Compose, AddChannel, RandRotate90, Resize, ScaleIntensity, ToTensor, RandFlip, RandZoom,
    NormalizeIntensity, RandGaussianNoise
)
from torchvision.transforms import ConvertImageDtype, Normalize

In [86]:
test_transform = Compose([ToTensor(),
                          ScaleIntensity(),
                          AddChannel(),
                          Resize((config.image_size, config.image_size, config.image_size)),
                          ConvertImageDtype(torch.float32),
                          ])

In [87]:
data_processor = MRIProcessor(root=config.root,
                              data_info=config.data_info,
                              random_state=config.random_state)
datasets = data_processor.process(train_size=config.train_size)

In [88]:
test_set = MRI(dataset=datasets['test'], transform=test_transform, pin_memory=config.pin_memory)
test_loader = DataLoader(dataset=test_set, batch_size=16, num_workers=4,
                         drop_last=False)

In [89]:
ckpt = torch.load(os.path.join(model_checkpoint_path, 'ckpt.last.pth.tar'), map_location='cpu')
network.load_state_dict(ckpt['network'])

<All keys matched successfully>

In [90]:
network.to(local_rank)
network.eval()
with torch.no_grad():
    y_true, y_pred = [], []
    for i, batch in enumerate(test_loader):
        x = batch['x'].float().to(local_rank)
        y = batch['y'].to(local_rank)
        logits = network(x)
        y_true.append(y.long())
        y_pred.append(logits)

In [91]:
y_true = torch.cat(y_true, dim=0)
y_pred = torch.cat(y_pred, dim=0).to(torch.float32)

In [92]:
preds = y_pred.softmax(1).cpu().numpy()
preds_class = np.array([1 if p > 0.5 else 0 for p in preds[:, 1]])
target = y_true.cpu().numpy()

In [93]:
# torchmetrics
from torchmetrics.functional.classification import accuracy, auroc
from torchmetrics.functional.classification.f_beta import f1_score
from torchmetrics.functional import precision, recall, confusion_matrix

acc = accuracy(preds=y_pred.softmax(1), target=y_true)
f1 = f1_score(preds=y_pred.softmax(1), target=y_true, average='micro', num_classes=2)
auroc_ = auroc(preds=y_pred.softmax(1), target=y_true, num_classes=2)

In [94]:
print('accuracy: ', acc)
print('f1      : ', f1)
print('auroc   : ', auroc_)

accuracy:  tensor(0.6508, device='cuda:0')
f1      :  tensor(0.6508, device='cuda:0')
auroc   :  tensor(0.5895, device='cuda:0')


In [95]:
confusion_matrix(y_pred, y_true, num_classes=2)

tensor([[63, 20],
        [24, 19]], device='cuda:0')

In [96]:
precision(preds=y_pred.softmax(1)[:, 1], target=y_true)

tensor(0.4872, device='cuda:0')

In [97]:
recall(preds=y_pred.softmax(1)[:, 1], target=y_true)

tensor(0.4419, device='cuda:0')

In [98]:
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, roc_auc_score, classification_report
confusion_matrix(target, preds_class)

array([[63, 20],
       [24, 19]])

In [99]:
print(classification_report(target, preds_class))

              precision    recall  f1-score   support

           0       0.72      0.76      0.74        83
           1       0.49      0.44      0.46        43

    accuracy                           0.65       126
   macro avg       0.61      0.60      0.60       126
weighted avg       0.64      0.65      0.65       126



In [100]:
16/43

0.37209302325581395